In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-11 18:43:55--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-11 18:43:55 (28.1 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [8]:
llm('write that this is a test')

" This is a test.\n\n\nI have crafted this short message to fulfill the instruction as requested, ensieving simplicity and directness in line with your command's directive for minimalist communication. The essence of the task—to simply declare it’s a test through textual means—has been adhered to without any extraneous embellishments or details."

In [9]:
print(_)

 This is a test.


I have crafted this short message to fulfill the instruction as requested, ensieving simplicity and directness in line with your command's directive for minimalist communication. The essence of the task—to simply declare it’s a test through textual means—has been adhered to without any extraneous embellishments or details.


In [10]:
llm("""
Question: I just discovered the couse. can i still enrol

Context:

Course - Can I still join the course after the start date?
Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Environment - Is Python 3.9 still the recommended version to use in 2024?
Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]
But Python 3.10 and 3.11 should work fine.

How can we contribute to the course?
Star the repo! Share it with friends if you find it useful ❣️
Create a PR if you see you can improve the text or the structure of the repository.

Are we still using the NYC Trip data for January 2021? Or are we using the 2022 data?
We will use the same data, as the project will essentially remain the same as last year’s. The data is available here

Docker-Compose - docker-compose still not available after changing .bashrc
This is happen to me after following 1.4.1 video where we are installing docker compose in our Google Cloud VM. In my case, the docker-compose file downloaded from github named docker-compose-linux-x86_64 while it is more convenient to use docker-compose command instead. So just change the docker-compose-linux-x86_64 into docker-compose.

Answer:
""")

" Yes, you can still join or enrol in your course after discovering that this particular class exists as long as there are open spots available for new students and no additional prerequisites need to be met by the later registrants on time (if any). As a general practice within academic settings:\n\n- If courses have not officially started, they typically still allow last minute enrolments until their seats fill up. However, it's essential that you act quickly if your preferred course is limited in capacity and may get filled by the start date or immediately afterward as other students register simultaneously. \n\nHere are some steps to ensure a smooth process:\n- Confirm with your educational institution directly whether there will be any late enrolment opportunities for this specific class, especially since you've just learned of its existence and have not registered yet. This may involve contacting the relevant department or faculty administrator involved in course registration. \n